In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2-2b-it.gguf",
                                         from_type="gemma2",
                                         load_eager=True
                                         )

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.icl_sfc_utils_gemma_2 import Circuitizer

In [6]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [7]:
task_names = ["en_es", "antonyms", "person_profession"]
task_name = task_names[0]
task_name = "antonyms"

In [8]:
%env JAX_TRACEBACK_FILTERING=off
import jax
jax.config.update('jax_traceback_filtering', 'off')

env: JAX_TRACEBACK_FILTERING=off


In [10]:
def check_if_single_token(token):
    return len(tokenizer.tokenize(token)) == 1

task = tasks[task_name]

print(len(task))

# task = {
#     k:v for k,v in task.items() if check_if_single_token(k) and check_if_single_token(v)
# }

print(len(task))

pairs = list(task.items())

batch_size = 8 
n_shot=16
if task_name.startswith("algo"):
    n_shot = 8
max_seq_len = 128
seed = 10

prompt = "Follow the pattern:\n{}"

runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max_seq_len, seed=seed, prompt=prompt, use_same_examples=False, use_same_target=False)


layers = list(range(15, 20))
circuitizer = Circuitizer(llama, tokenizer, runner, layers, prompt)

# circuitizer.run_metrics()

162
162
Setting up masks...
Running metrics...
Setting up RMS...


  0%|          | 0/26 [00:00<?, ?it/s]

ValueError: The following fields were not initialised during __init__: {'sae_error_attn', 'ie_attn', 'saes', 'ie_error_transcoder', 'ie_transcoder', 'sae_grads_resid', 'ie_resid', 'ie_error_attn', 'ie_error_resid', 'sae_error_resid', 'sae_error_transcoder', 'sae_grads_transcoder', 'sae_grads_attn'}

In [21]:
from micrlhf.utils.load_sae import get_dm_res_sae

sae = get_dm_res_sae(18, load_65k=True, type="att")

In [23]:
sae["W_dec"].shape

(65536, 2048)

In [25]:
from micrlhf.llama import LlamaAttention

get_resids = llama.select().at_instances_of(LlamaAttention).apply_with_selected_index(lambda l, b: b.select().at(lambda x: x.attn_value_to_output).at_instances_of(pz.nn.Linear).apply_with_selected_index(lambda i, x: pz.nn.Sequential([
    pz.de.TellIntermediate.from_config(tag=f"resid_attn_{l}"),
    x,
])))

In [26]:
train_tokens = runner.get_tokens(
    runner.train_pairs, tokenizer
)["input_ids"]

tokens_wrapped = pz.nx.wrap(train_tokens, "batch", "seq")

llama_inputs = llama.inputs.from_basic_segments(tokens_wrapped)

In [27]:
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: True)

In [28]:
logits, resids = get_resids(llama_inputs)

In [33]:
resids[5].value.untag("kv_heads", "q_rep", "projection").reshape(-1).tag("embedding").unwrap(
    "batch", "seq", "embedding"
)

<jax.Array bfloat16(8, 128, 2048) ≈0.0027 ±0.42 [≥-4.9, ≤3.8] nonzero:2_097_152>

In [35]:
resids[5].value

<NamedArray bfloat16(| batch:8, seq:128, kv_heads:4, q_rep:2, projection:256) ≈0.0027 ±0.42 [≥-4.9, ≤3.8] nonzero:2_097_152 (wrapping jax.Array)>

In [34]:
llama.config

LlamaConfig(vocab_size=256000, hidden_size=2304, intermediate_size=9216, num_attention_heads=8, num_key_value_heads=4, head_dim=256, num_layers=26, parameter_dtype=<class 'jax.numpy.bfloat16'>, activation_dtype=<class 'jax.numpy.bfloat16'>, act_fn='gelu', resid_rescale=Array(48, dtype=bfloat16), attn_soft_cap=None, final_soft_cap=None, attn_scale_dim=288, pre_post_ln=True)

In [ ]:
from micrlhf.llama import LlamaMLP

llama.select().at_instances_of(LlamaMLP).pick_nth_selected(18).get()

LlamaMLP( # Sequential
  sublayers=[
    BranchAndMultiplyTogether(
      branches=[NamedGroup(name='gate', sublayers=[Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2304, neurons:9216) ≈-2.6e-05 ±0.0078 [≥-0.11, ≤0.12] zero:161_052 nonzero:21_072_612 (wrapping jax.Array)>, name='blocks.18.mlp.gate_proj.weights'), in_axis_names=('embedding',), out_axis_names=('neurons',)), Elementwise(fn=jax.nn.gelu)]), Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2304, neurons:9216) ≈8.2e-07 ±0.0077 [≥-0.097, ≤0.11] zero:162_080 nonzero:21_071_584 (wrapping jax.Array)>, name='blocks.18.mlp.up_proj.weights'), in_axis_names=('embedding',), out_axis_names=('neurons',))],
    ),
    Linear(
      #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
      # Input: <ArraySpec floating(| **{B:…}, neurons:9216)>
      # Output: <ArraySpec floating(| **{B:…}, embedding:2304)>
      #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
      weights=Parameter(value=<NamedArray bfloat16(| neurons:9216, embedding:2304) ≈8e-07 ±0.0074 [≥-0.25, ≤0.28] zero:164_728 nonzero:21_068_936 (wrapping jax.Array)>, name='blocks.18.mlp.out_proj.weights'),
      in_axis_names=('neurons',),
      out_axis_names=('embedding',),
    ),
  ],
)

In [13]:
from micrlhf.llama import LlamaBlock

block = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(18).get()

In [14]:
block

LlamaBlock( # Sequential
  #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
  # Broken handler refs: [HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_cd5b7', tag='positions'), HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_cd5b7', tag='attn_mask')]
  #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
  sublayers=[
    WithConstantSideInputsNonPytree(
      handler_id='WithConstantSideInputsNonPytree_febf8',
      body=ConstrainedSharding(mesh=HandledSideInputRef(handler_id='WithConstantSideInputsNonPytree_febf8', tag='mesh'), axis_name_to_mesh_name=HandledSideInputRef(handler_id='WithConstantSideInputsNonPytree_febf8', tag='axis_name_to_mesh_name')),
      side_inputs={'axis_name_to_mesh_name': {'neurons': 'mp', 'kv_heads': 'mp', 'seq': 'sp', 'batch': 'dp', 'mp': 'mp', 'dp': 'dp', 'sp': 'sp'}, 'mesh': <jax._src.mesh.Mesh object at 0x7f959c5b3ee0>},
    ),
    Residual(
      delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(9.98378e-07, dtype=bfloat16)>), Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2304) ≈1.6 ±0.42 [≥0.78, ≤3.2] nonzero:2_304 (wrapping jax.Array)>, name='blocks.18.pre_attn_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), CastToDType(dtype=jax.numpy.bfloat16), LlamaAttention(input_to_query=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2304, kv_heads:4, q_rep:2, projection:256) ≈-8e-06 ±0.013 [≥-0.09, ≤0.13] zero:37_965 nonzero:4_680_627 (wrapping jax.Array)>, name='blocks.18.attn.query.weights'), in_axis_names=('embedding',), out_axis_names=('kv_heads', 'q_rep', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_cd5b7', tag='positions')), ConstantRescale(by=<jax.Array(0.0588379, dtype=bfloat16)>)]), input_to_key=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2304, kv_heads:4, projection:256) ≈-1e-07 ±0.012 [≥-0.12, ≤0.11] zero:18_414 nonzero:2_340_882 (wrapping jax.Array)>, name='blocks.18.attn.key.weights'), in_axis_names=('embedding',), out_axis_names=('kv_heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_cd5b7', tag='positions')), CastToDType(dtype=jax.numpy.bfloat16)]), input_to_value=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2304, kv_heads:4, projection:256) ≈-6.1e-06 ±0.011 [≥-0.086, ≤0.088] zero:21_278 nonzero:2_338_018 (wrapping jax.Array)>, name='blocks.18.attn.value.weights'), in_axis_names=('embedding',), out_axis_names=('kv_heads', 'projection')), CastToDType(dtype=jax.numpy.bfloat16)]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'kv_heads': 'h', 'q_rep': 'r', 'projection': 'p'}, {'seq': 'tkv', 'kv_heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'kv_heads': 'h', 'q_rep': 'r', 'kv_seq': 'tkv'}), ApplyAttentionMask(mask=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_cd5b7', tag='attn_mask'), masked_out_value=<jax.Array(-3.38953e+38, dtype=bfloat16)>), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'kv_heads': 'h', 'q_rep': 'r', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'kv_heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'kv_heads': 'h', 'q_rep': 'r', 'projection': 'p'}), Linear(weights=Parameter(value=<NamedArray bfloat16(| kv_heads:4, q_rep:2, projection:256, embedding:2304) ≈-4.3e-06 ±0.011 [≥-0.38, ≤0.34] zero:35_834 nonzero:4_682_758 (wrapping jax.Array)>, name='blocks.18.attn.output.weights'), in_axis_names=('kv_heads', 'q_rep', 'projection'), out_axis_names=('embedding',))])), RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(9.98378e-07, dtype=bfloat16)>), Linear(weights=Parameter(value=<NamedArray bfloat16(| embeddin

In [12]:
llama

KeyboardInterrupt: 

In [ ]:
from micrlhf.llama import LlamaTransformer
llama_1 = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [ ]:
llama_1

LlamaTransformer(
  #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
  # Input: LlamaInputs(tokens=<tokens>, attention_mask=<attention mask>, positions=<positions>)
  # Output: <unnormalized logits>
  #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
  config=LlamaConfig(vocab_size=256000, hidden_size=2048, intermediate_size=16384, num_attention_heads=8, num_key_value_heads=1, head_dim=256, num_layers=18, parameter_dtype=jax.numpy.bfloat16, activation_dtype=jax.numpy.bfloat16, act_fn='gelu', resid_rescale=<jax.Array(45.25, dtype=bfloat16)>, attn_soft_cap=None, final_soft_cap=None, attn_scale_dim=None, pre_post_ln=False),
  body=WithSideInputsFromInputTuple(handler_id='WithSideInputsFromInputTuple_a9559', body=Sequential(sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=Parameter(value=<NamedArray bfloat16(| vocabulary:256000, embedding:2048) ≈0.0016 ±0.18 [≥-9.8, ≤6.0] zero:5_685_388 nonzero:518_602_612 (wrapping jax.Array)>, name='embed.embeddings'), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(45.25, dtype=bfloat16)>), WithConstantSideInputsNonPytree(handler_id='WithConstantSideInputsNonPytree_febf8', body=ConstrainedSharding(mesh=HandledSideInputRef(handler_id='WithConstantSideInputsNonPytree_febf8', tag='mesh'), axis_name_to_mesh_name=HandledSideInputRef(handler_id='WithConstantSideInputsNonPytree_febf8', tag='axis_name_to_mesh_name')), side_inputs={'axis_name_to_mesh_name': {'neurons': 'mp', 'kv_heads': 'mp', 'seq': 'sp', 'batch': 'dp', 'mp': 'mp', 'dp': 'dp', 'sp': 'sp'}, 'mesh': Mesh(device_ids=array([[[0]]]), axis_names=('dp', 'sp', 'mp'))}), Sequential(sublayers=[LlamaBlock(sublayers=[WithConstantSideInputsNonPytree(handler_id='WithConstantSideInputsNonPytree_febf8', body=ConstrainedSharding(mesh=HandledSideInputRef(handler_id='WithConstantSideInputsNonPytree_febf8', tag='mesh'), axis_name_to_mesh_name=HandledSideInputRef(handler_id='WithConstantSideInputsNonPytree_febf8', tag='axis_name_to_mesh_name')), side_inputs={'axis_name_to_mesh_name': {'neurons': 'mp', 'kv_heads': 'mp', 'seq': 'sp', 'batch': 'dp', 'mp': 'mp', 'dp': 'dp', 'sp': 'sp'}, 'mesh': Mesh(device_ids=array([[[0]]]), axis_names=('dp', 'sp', 'mp'))}), Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(9.98378e-07, dtype=bfloat16)>), Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2048) ≈2.6 ±1.4 [≥-0.27, ≤9.8] zero:9 nonzero:2_039 (wrapping jax.Array)>, name='blocks.0.pre_attn_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), CastToDType(dtype=jax.numpy.bfloat16), LlamaAttention(input_to_query=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2048, kv_heads:1, q_rep:8, projection:256) ≈2.2e-06 ±0.0086 [≥-0.21, ≤0.18] zero:68_150 nonzero:4_126_154 (wrapping jax.Array)>, name='blocks.0.attn.query.weights'), in_axis_names=('embedding',), out_axis_names=('kv_heads', 'q_rep', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a9559', tag='positions')), ConstantRescale(by=<jax.Array(0.0625, dtype=bfloat16)>)]), input_to_key=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2048, kv_heads:1, projection:256) ≈-5.5e-06 ±0.012 [≥-0.098, ≤0.1] zero:9_918 nonzero:514_370 (wrapping jax.Array)>, name='blocks.0.attn.key.weights'), in_axis_names=('embedding',), out_axis_names=('kv_heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a9559', tag='positions')), CastToDType(dtype=jax.numpy.bfloat16)]), input_to_value=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray bfloat16(| embedding:2048, kv_heads:1, projection:256) ≈5.4e-05 ±0.017 [≥-0.25, ≤0.23] zero:13_577 nonzero:510_711 (wrapping jax.Array)>, name='blocks.0.attn.value.weights'), in_axis_names=('embedding',), out_axis_names=('